In [1]:
import requests
import pandas as pd
import numpy as np
import geopandas as gpd
from geopandas import GeoDataFrame
import requests
import json
from bs4 import BeautifulSoup as BS
from IPython.core.display import HTML
import re
import time 
from tqdm.notebook import tqdm

### An Analysis of Political Contributions During the 2020 House of Representatives Election

#### In this part, you will obtain as much data as you can on the campaign contributions received by each candidate. This data is avaiable through the website https://www.opensecrets.org/. At the end of the project, your group will give a presentation of your findings.



#### 1. Start by scraping the data from the summary page for Tennessee's 2nd District, which is available at https://www.opensecrets.org/races/summary?cycle=2020&id=TN02&spec=N.
- The data that we want is contained in the "Total Raised and Spent" table.
- Make a DataFrame showing, for each candidate:
    - the candidate's name
    - the candidate's party
    - state
    - district number
    - whether the candidate was an incumbent
    - whether the candidate won the race
    - the total amount raised by that candidate (as a numeric variable)
    - the total amount spent by the candidate (as a numeric variable)

In [2]:
URL = 'https://www.opensecrets.org/races/summary?cycle=2020&id=TN02&spec=N'

response = requests.get(URL)
#type(response)
#response.status_code

In [3]:
requests.get('https://www.opensecrets.org/races/summary?cycle=2020&id=TN02&spec=N')

<Response [200]>

In [4]:
#response.text
soup = BS(response.text)
#print(soup.prettify())

In [5]:
#reading the data into a dataframe 
TN_02=pd.read_html(str(soup.find('table')))[0]
TN_02

,Candidate,Raised,Spent,Cash on Hand,Last Report
0,Tim Burchett (R) • Incumbent • Winner,"$1,336,276","$878,488","$593,678",12/31/2020
1,Renee Hoyos (D),"$812,784","$816,793",$210,12/31/2020


In [6]:
#keeping only the columns we want and deleting the rest 
TN_02= TN_02[['Candidate', 'Raised', 'Spent']]
TN_02

,Candidate,Raised,Spent
0,Tim Burchett (R) • Incumbent • Winner,"$1,336,276","$878,488"
1,Renee Hoyos (D),"$812,784","$816,793"


In [7]:
TN_02 ['Candidate']= TN_02 ['Candidate'].astype(str)

<ipython-input-7-bdfcb1db691f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TN_02 ['Candidate']= TN_02 ['Candidate'].astype(str)


In [8]:
#splitting the candidate column into names, incumbent, winner
TN_02 [['Candidate', 'Incumbent', 'Winner']] = TN_02['Candidate'].str.split('•', 2, expand = True)

/Users/vahidehrasekhi/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [9]:
#splitting candidate column into names and party 
TN_02 [['Candidate','Party']] = TN_02['Candidate'].str.split('(', expand = True)
TN_02

,Candidate,Raised,Spent,Incumbent,Winner,Party
0,Tim Burchett,"$1,336,276","$878,488",Incumbent,Winner,R)
1,Renee Hoyos,"$812,784","$816,793",None,None,D)


In [10]:
#removing parenthesis after the R and D in party column 
TN_02['Party']=TN_02['Party'].str.replace (')', '')
TN_02

<ipython-input-10-efda2b01e9ff>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  TN_02['Party']=TN_02['Party'].str.replace (')', '')


,Candidate,Raised,Spent,Incumbent,Winner,Party
0,Tim Burchett,"$1,336,276","$878,488",Incumbent,Winner,R
1,Renee Hoyos,"$812,784","$816,793",None,None,D


#### 2. Once you have working code for Tennessee's 2nd District, expand on your code to capture all of Tennessee's districts.

In [11]:
#creating a function to get data from other districts of TN
def district_states (state, district):

    response = requests.get('https://www.opensecrets.org/races/summary?cycle=2020&id=TN02&spec=N')
    soup = BS(response.text)
    candidates = pd.read_html(str(soup.find('table')))[0]
    candidates['state']= state
    candidates['district']= district
    
    return candidates

In [12]:
district_states ('TN', '02')

,Candidate,Raised,Spent,Cash on Hand,Last Report,state,district
0,Tim Burchett (R) • Incumbent • Winner,"$1,336,276","$878,488","$593,678",12/31/2020,TN,02
1,Renee Hoyos (D),"$812,784","$816,793",$210,12/31/2020,TN,02


In [13]:
district_states ('TN', '01')

,Candidate,Raised,Spent,Cash on Hand,Last Report,state,district
0,Tim Burchett (R) • Incumbent • Winner,"$1,336,276","$878,488","$593,678",12/31/2020,TN,01
1,Renee Hoyos (D),"$812,784","$816,793",$210,12/31/2020,TN,01


In [14]:
district_states ('TN', '03')

,Candidate,Raised,Spent,Cash on Hand,Last Report,state,district
0,Tim Burchett (R) • Incumbent • Winner,"$1,336,276","$878,488","$593,678",12/31/2020,TN,03
1,Renee Hoyos (D),"$812,784","$816,793",$210,12/31/2020,TN,03


In [15]:
# The above code works but we need to chnage it to make sure the table gives us only the data we are interested in 
#and the forrmat we want.

#### 3. Once you have working code for all of Tennessee's districts, expand on it to capture all states and districts. The number of representatives each state has can be found in a table on this page: https://www.britannica.com/topic/United-States-House-of-Representatives-Seats-by-State-1787120

In [16]:
#getting the data from britannica for all states and districts 
URL = 'https://www.britannica.com/topic/United-States-House-of-Representatives-Seats-by-State-1787120'

response = requests.get(URL)
#type(response)
#response.status_code

In [17]:
requests.get('https://www.britannica.com/topic/United-States-House-of-Representatives-Seats-by-State-1787120')
soup = BS(response.text)

In [18]:
#getting the table of states and dropping th3 50th row which is just the total
state_rep=pd.read_html(str(soup.find('table')))[0]
state_rep= state_rep[0:50] 
state_rep

,state,representatives
0,Alabama,7
1,Alaska,1
2,Arizona,9
3,Arkansas,4
4,California,53
5,Colorado,7
6,Connecticut,5
7,Delaware,1
8,Florida,27
9,Georgia,14


In [19]:
#It would be easier to work with the data if we have abbreviations for the states in the table above. 
#So, we can Webscrape https://worldpopulationreview.com/states/state-abbreviations

In [20]:
abbrevation_url = 'https://worldpopulationreview.com/states/state-abbreviations'
abbrevation_url_response = requests.get(abbrevation_url)
response

<Response [200]>

In [21]:
soup = BS(abbrevation_url_response.text)

abbrevations = pd.read_html(str(soup.find('table')))[0]
abbrevations

,State,Abbreviation,Code
0,Alabama,Ala.,AL
1,Alaska,Alaska,AK
2,Arizona,Ariz.,AZ
3,Arkansas,Ark.,AR
4,California,Calif.,CA
5,Colorado,Colo.,CO
6,Connecticut,Conn.,CT
7,Delaware,Del.,DE
8,District of Columbia,D.C.,DC
9,Florida,Fla.,FL


In [22]:
districts = state_rep.merge(abbrevations,
                           left_on = 'state',
                           right_on = 'State')
districts

,state,representatives,State,Abbreviation,Code
0,Alabama,7,Alabama,Ala.,AL
1,Alaska,1,Alaska,Alaska,AK
2,Arizona,9,Arizona,Ariz.,AZ
3,Arkansas,4,Arkansas,Ark.,AR
4,California,53,California,Calif.,CA
5,Colorado,7,Colorado,Colo.,CO
6,Connecticut,5,Connecticut,Conn.,CT
7,Delaware,1,Delaware,Del.,DE
8,Florida,27,Florida,Fla.,FL
9,Georgia,14,Georgia,Ga.,GA


In [23]:
#dropping the State columns
districts = districts.drop(columns = 'State')

In [24]:
#dropping the Abbreviation column
districts = districts.drop(columns = 'Abbreviation')

In [25]:
#changing the TN url to get the data from different states and districts 
def get_district_candidates (state, district):

    response = requests.get('https://www.opensecrets.org/races/summary?cycle=2020&id=' +state + district + '&spec=N')
    soup = BS(response.text)
    candidates = pd.read_html(str(soup.find('table')))[0]
    candidates['state']= state
    candidates['district']= district
    
    return candidates 

In [26]:
get_district_candidates ('TN', '04')

,Candidate,Raised,Spent,Cash on Hand,Last Report,state,district
0,Scott Desjarlais (R) • Incumbent • Winner,"$331,464","$392,499","$302,649",12/31/2020,TN,04
1,Christopher Hale (D),"$308,731","$302,996","$5,735",12/31/2020,TN,04


In [27]:
get_district_candidates ('TX', '03')

,Candidate,Raised,Spent,Cash on Hand,Last Report,state,district
0,Van Taylor (R) • Incumbent • Winner,"$2,655,327","$2,825,378","$22,498",12/31/2020,TX,03
1,Helane Seikaly (D),"$1,635,185","$1,634,843",$342,12/31/2020,TX,03


In [28]:
get_district_candidates ('CA', '04')

,Candidate,Raised,Spent,Cash on Hand,Last Report,state,district
0,Tom McClintock (R) • Incumbent • Winner,"$2,053,363","$2,107,248","$171,001",12/31/2020,CA,04
1,Brynne Kennedy (D),"$3,006,020","$3,002,532","$3,489",12/31/2020,CA,04


In [ ]:
#The above codes work but we have to repeat it for each state and district. So, we can write a function to avoid this issue.

In [29]:
state_dist = pd.DataFrame()

for index in tqdm(districts.index):
    state = districts['Code'][index]
    dist_no = districts['representatives'][index]
    
    if index == 10:
        time.sleep(70)
    elif index == 20:
        time.sleep(70)
    elif index == 30:
        time.sleep(70)
    elif index == 40:
        time.sleep(70)
    
    for i in range(1, dist_no + 1):
        if i < 10:
            i = '0' + str(i)
        state_dist = state_dist.append(get_district_candidates(str(state), str(i)),
                                      ignore_index = True)
      

  0%|          | 0/50 [00:00<?, ?it/s]

In [30]:
state_dist.to_csv('../data/all_state.csv',
                 index = False)

In [31]:
all_state= pd.read_csv('../data/all_state.csv')
all_state.head()

,Candidate,Raised,Spent,Cash on Hand,Last Report,state,district
0,Jerry Carl (R) • Winner,"$1,971,321","$1,859,349","$111,973",12/31/2020,AL,1
1,James Averhart (D),"$80,095","$78,973","$1,122",12/31/2020,AL,1
2,Barry Moore (R) • Winner,"$650,807","$669,368","-$13,633",12/31/2020,AL,2
3,Phyllis Harvey-Hall (D),"$56,050","$55,988",$0,12/31/2020,AL,2
4,Mike D Rogers (R) • Incumbent • Winner,"$1,193,111","$1,218,564","$502,234",12/31/2020,AL,3


In [32]:
all_state.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 894 entries, 0 to 893
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Candidate     894 non-null    object
 1   Raised        894 non-null    object
 2   Spent         894 non-null    object
 3   Cash on Hand  894 non-null    object
 4   Last Report   894 non-null    object
 5   state         894 non-null    object
 6   district      894 non-null    int64 
dtypes: int64(1), object(6)
memory usage: 49.0+ KB


In [33]:
all_state['Cash on Hand']= all_state['Cash on Hand'].str.replace (r'\$|,', '', regex=True).astype('int')

In [34]:
all_state['Raised']= all_state['Raised'].str.replace (r'\$|,', '', regex=True).astype('int')

In [35]:
all_state['Spent']= all_state['Spent'].str.replace (r'\$|,', '', regex=True).astype('int')

In [36]:
all_state.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 894 entries, 0 to 893
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Candidate     894 non-null    object
 1   Raised        894 non-null    int64 
 2   Spent         894 non-null    int64 
 3   Cash on Hand  894 non-null    int64 
 4   Last Report   894 non-null    object
 5   state         894 non-null    object
 6   district      894 non-null    int64 
dtypes: int64(4), object(3)
memory usage: 49.0+ KB


In [37]:
#changing the data type
all_state = all_state.astype({
    'Raised':'float64',
    'Spent':'float64',
    'Cash on Hand':'float64',
    'Last Report':'datetime64'
})

In [38]:
all_state[['Candidate', 'Incumbent', 'Winner']] = all_state['Candidate'].str.split('•', 2, expand = True)
all_state.head()

,Candidate,Raised,Spent,Cash on Hand,Last Report,state,district,Incumbent,Winner
0,Jerry Carl (R),1971321.0,1859349.0,111973.0,2020-12-31,AL,1,Winner,None
1,James Averhart (D),80095.0,78973.0,1122.0,2020-12-31,AL,1,None,None
2,Barry Moore (R),650807.0,669368.0,-13633.0,2020-12-31,AL,2,Winner,None
3,Phyllis Harvey-Hall (D),56050.0,55988.0,0.0,2020-12-31,AL,2,None,None
4,Mike D Rogers (R),1193111.0,1218564.0,502234.0,2020-12-31,AL,3,Incumbent,Winner


In [39]:
#splitting candidate column into names and party 
all_state[['Candidate','Party']] = all_state['Candidate'].str.split('(', 1, expand = True)

In [40]:
#removing parenthesis after the R and D in party column 
all_state['Party']=all_state['Party'].str.replace (')', '')

<ipython-input-40-af6aca3efb8c>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  all_state['Party']=all_state['Party'].str.replace (')', '')


In [41]:
all_state = (all_state.rename({'Candidate':'name',
                 'Raised': 'raised',
                 'Spent':'spent',
                  'Cash on Hand': 'cash_left',
                  'Incumbent': 'incumbent',
                  'Winner': 'winner',
                  'Party': 'party',
                   'Last Report': 'year'},
                   axis='columns'))

In [42]:
all_state.head()

,name,raised,spent,cash_left,year,state,district,incumbent,winner,party
0,Jerry Carl,1971321.0,1859349.0,111973.0,2020-12-31,AL,1,Winner,None,R
1,James Averhart,80095.0,78973.0,1122.0,2020-12-31,AL,1,None,None,D
2,Barry Moore,650807.0,669368.0,-13633.0,2020-12-31,AL,2,Winner,None,R
3,Phyllis Harvey-Hall,56050.0,55988.0,0.0,2020-12-31,AL,2,None,None,D
4,Mike D Rogers,1193111.0,1218564.0,502234.0,2020-12-31,AL,3,Incumbent,Winner,R


In [43]:
#all_state= all_state.drop(columns='none')
states_df=all_state.to_csv('../data/states_df.csv', index=False) 
#if we don't add index= False, we will have an addiitonal column 'unnamed'